In [ ]:
# Validate columns
    required_columns = ['Deal', 'True Need', 'Allotment', 'Dealer', 'Trader', 'Negotiated/Competitive', 'Date']
    missing_columns = [col for col in required_columns if col not in self.original_data.columns]
    
    if missing_columns:
        raise ValueError(f"Missing columns: {', '.join(missing_columns)}")
    
    # Preprocess the data
    self._preprocess_data()

def _preprocess_data(self):
    """
    Preprocess and clean the input data
    """
    # Convert Date column to datetime
    self.original_data['Date'] = pd.to_datetime(self.original_data['Date'])
    
    # Ensure numeric columns
    self.original_data['True Need'] = pd.to_numeric(self.original_data['True Need'], errors='coerce')
    self.original_data['Allotment'] = pd.to_numeric(self.original_data['Allotment'], errors='coerce')
    
    # Calculate Fill Rate
    self.original_data['Fill Rate (%)'] = np.where(
        self.original_data['True Need'] > 0,
        (self.original_data['Allotment'] / self.original_data['True Need']) * 100,
        0
    ).round(2)

def generate_deal_participation_log(self):
    """
    Generate the Deal Participation Log with formatted columns
    """
    # Create a copy of the data for formatting
    df = self.original_data.copy()
    
    # Format numeric columns
    df['True Need'] = df['True Need'].apply(lambda x: f'${x:,.2f}')
    df['Allotment'] = df['Allotment'].apply(lambda x: f'${x:,.2f}')
    df['Fill Rate (%)'] = df['Fill Rate (%)'].apply(lambda x: f'{x:.2f}%')
    
    # Reorder columns for better readability
    columns = [
        'Date', 'Deal', 'Dealer', 'Trader', 
        'True Need', 'Allotment', 'Fill Rate (%)', 
        'Negotiated/Competitive'
    ]
    
    return df[columns]

def generate_dealer_performance_summary(self):
    """
    Generate Dealer Performance Summary with formatted columns
    """
    # Group and aggregate data
    dealer_summary = self.original_data.groupby('Dealer').agg({
        'Deal': 'count',  # Total Deals Participated
        'True Need': 'sum',  # Total Requested Allocation
        'Allotment': 'sum',  # Total Actual Allocation
        'Fill Rate (%)': 'mean'  # Average Fill Rate
    }).reset_index()
    
    # Rename columns
    dealer_summary.columns = [
        'Dealer Name', 
        'Total Deals Participated', 
        'Total Requested Allocation', 
        'Total Actual Allocation', 
        'Average Fill Rate'
    ]
    
    # Format numeric columns
    dealer_summary['Total Requested Allocation'] = dealer_summary['Total Requested Allocation'].apply(lambda x: f'${x:,.2f}')
    dealer_summary['Total Actual Allocation'] = dealer_summary['Total Actual Allocation'].apply(lambda x: f'${x:,.2f}')
    dealer_summary['Average Fill Rate'] = dealer_summary['Average Fill Rate'].apply(lambda x: f'{x:.2f}%')
    
    # Calculate Performance Ranking
    dealer_summary['Performance Ranking'] = dealer_summary['Average Fill Rate'].rank(method='dense', ascending=False).astype(int)
    
    # Reorder and sort columns
    columns = [
        'Dealer Name', 
        'Total Deals Participated', 
        'Total Requested Allocation', 
        'Total Actual Allocation', 
        'Average Fill Rate',
        'Performance Ranking'
    ]
    
    return dealer_summary.sort_values('Performance Ranking')[columns]

def export_trackers(self, output_path='muni_bond_trackers.xlsx'):
    """
    Export generated trackers to Excel with formatting
    """
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        # Deal Participation Log sheet
        deal_log = self.generate_deal_participation_log()
        deal_log.to_excel(writer, sheet_name='Deal Participation Log', index=False)
        
        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Deal Participation Log']
        
        # Add a format for the header
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#D7E4BC',
            'border': 1
        })
        
        # Write the column headers with the defined format
        for col_num, value in enumerate(deal_log.columns.values):
            worksheet.write(0, col_num, value, header_format)
        
        # Dealer Performance Summary sheet
        dealer_summary = self.generate_dealer_performance_summary()
        dealer_summary.to_excel(writer, sheet_name='Dealer Performance Summary', index=False)
        
        # Format headers for Dealer Performance Summary
        worksheet = writer.sheets['Dealer Performance Summary']
        for col_num, value in enumerate(dealer_summary.columns.values):
            worksheet.write(0, col_num, value, header_format)
    
    print(f"Trackers exported to {output_path}")



  # Print out the Deal Participation Log
    print("\nDeal Participation Log (First 5 rows):")
    print(tracker.generate_deal_participation_log().head().to_string(index=False))
    
    # Print out the Dealer Performance Summary
    print("\nDealer Performance Summary:")
    print(tracker.generate_dealer_performance_summary().to_string(index=False))
    
    # Export trackers to Excel
    tracker.export_trackers()

except Exception as e:
    print(f"An error occurred: {e}")